In [1]:
!pip install -I -q folium==0.8.3
import pandas as pd
import folium
import geojson
import geopandas as gpd
from IPython.display import IFrame
import numpy as np

In [ ]:
# sanity check- should be 0.8.3
folium.__version__

In [2]:
# read in csv data
socio = pd.read_csv("IRIS_SOCIOLOGIE.csv", index_col=0)
socio.IRIS = socio.IRIS.astype(str) # convert IRIS var to str for correct joining with geojson
socio["test_col"] = np.random.rand(socio.shape[0]) # create dummy column for mapping example
socio.head()

,N_SQ_IR,C_CAINSEE,N_QU,C_IR,C_TYPEIR,L_IR,M2_IP,M2_POP,M2_EMP,NOM_EPT,...,TX12_MENPSEUL,C12_NE24F1,C12_NE24F2,C12_NE24F3,C12_NE24F4P,C12_NE24F,TX12_NE24F,SHAPE_Length,SHAPE_Area,test_col
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,750004511,75101,1.0,751010101,H,Saint-Germain L'Auxerrois 1,35395.547336,22312.521652,33459.689307,Paris,...,49.735155,63.267904,42.045799,4.106184,0.0,109.419887,21.830591,1083.988805,64488.144150,0.998969
2,750004559,75101,1.0,751010102,A,Saint-Germain l'Auxerrois 2,45971.817707,4728.312240,42976.731425,Paris,...,59.225319,6.546429,0.000000,2.548365,0.0,9.094794,10.764376,1057.130418,61981.155490,0.816294
3,750004497,75101,1.0,751010103,A,Saint-Germain L'Auxerrois 3,29246.550230,17775.054727,26203.942322,Paris,...,55.725401,26.024292,7.996127,8.883467,0.0,42.903886,21.277736,877.312123,46058.484053,0.745650
4,750004460,75101,1.0,751010104,A,Saint-Germain L'Auxerrois 4,181708.577351,26416.300456,109150.202714,Paris,...,40.000000,0.887340,0.000000,0.000000,0.0,0.887340,20.000000,2052.894266,224788.065372,0.983404
5,750004388,75101,1.0,751010105,D,Tuileries,225540.072976,11.599440,2736.060142,Paris,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,2112.531964,248874.958259,0.581529


In [3]:
geo_path = "iris-sociologie.geojson"

# load geojson
g = geojson.load(open("iris-sociologie.geojson"))

# load geojson in GeoDataFrame (for EDA)
geo = gpd.read_file(geo_path)
geo.head()

,objectid,c12_ne24f,iris_num,libcom,m2_pop,c_cainsee,m2_ip,c12_ne24f1,c12_ne24f2,c12_ne24f3,...,dep,ept,n_sq_ir,m2_emp,c12_men,c12_menpse,com,shape_leng,n_qu,geometry
0,2037,430.184059,930510404,Noisy-le-Grand,120993.579796,93051,137986.307600,148.675220,190.210682,66.793863,...,93,T9,930000981,133162.713781,975.314552,287.794286,93051,1683.142955,NaN,"POLYGON ((2.572773194497697 48.84586444134267,..."
1,2041,511.742349,930510503,Noisy-le-Grand,286430.403739,93051,393649.312714,227.282832,191.407556,71.129112,...,93,T9,930003667,331823.948220,1080.387551,234.761237,93051,3075.628419,NaN,"POLYGON ((2.573775302054818 48.83983035309186,..."
2,2045,347.148124,930530104,Noisy-le-Sec,132735.121888,93053,143986.385543,189.523698,113.045499,34.016199,...,93,T8,930000989,143986.385543,1106.733656,444.188995,93053,1991.207560,NaN,"POLYGON ((2.469557509958308 48.89393826496879,..."
3,2046,449.647047,930530105,Noisy-le-Sec,104480.782672,93053,125715.037380,172.645048,166.811013,91.755655,...,93,T8,930000990,121669.843590,951.608874,287.454201,93053,1626.558112,NaN,"POLYGON ((2.46101714440833 48.89320595854011, ..."
4,2073,676.255070,930550701,Pantin,134165.605307,93055,219953.235720,344.383860,266.850632,55.000554,...,93,T8,930002374,208554.795192,1998.140953,818.331324,93055,3464.308809,NaN,"POLYGON ((2.409715918366365 48.89499550371073,..."


In [5]:
# create simple map with geojson outlines of arondissmonts
m1 = folium.Map(location=[48.8, 2.3], zoom_start=11)

folium.GeoJson(g).add_to(m1)

m1.save('images/simple-map.html') # save map to html file (Google Chrome bug workaround)
IFrame('images/simple-map.html', width='100%', height='750px') # display saved html file

# click on the whitespace to the left of the map to toggle scrolling


In [ ]:
# list of column names for choropleth reference
socio.columns

In [38]:
# create a choropleth overlay for C12_MEN variable
m2 = folium.Map(location=[48.8, 2.3], zoom_start=11)

folium.Choropleth(
 geo_data=g, # geojson 
 data=socio, # df
 columns=['IRIS', 'test_col'],   # first col must be IRIS, second col sets colors for overlay
 key_on='feature.properties.iris',   # key to match csv data with geojson areas
 fill_color='YlOrRd',
 bins=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0]).add_to(m2)

folium.LayerControl().add_to(m2)

m2.save('images/choropleth-map.html') # save map to html file (Google Chrome bug workaround)
IFrame('images/choropleth-map.html', width='100%', height='750px') # display saved html file

# click on the whitespace to the left of the cell to toggle scrolling
